Enron email sender classification

In [1]:
import pandas as pd

from spacy import load
from string import punctuation
from collections import Counter
from operator import itemgetter
from statistics import stdev
#import features

Import the data from the csv file to dataframe

In [2]:
emails_df = pd.read_csv('extracted-2.csv')
emails_df.head(1)

C:\Users\faraa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,file_id,msg,Message-ID,Date,From,To,Subject,Mime-Version,...,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,0,0,lokay-m/lokay-m_65/,Message-ID: <11819236.1075844017427.JavaMail.e...,<11819236.1075844017427.JavaMail.evans@thyme>,"Wed, 14 Jun 2000 01:24:00 -0700 (PDT)",frozenset({'steven.harris@enron.com'}),frozenset({'michele.lokay@enron.com'}),"Reminder: WEFA Meeting Tomorrow Morning, June ...",1.0,...,7bit,Steven Harris,Michele Lokay,NaN,NaN,\Michelle_Lokay_Dec2000_June2001_1\Notes Folde...,LOKAY-M,mlokay.nsf,FYI\n---------------------- Forwarded by Steve...,lokay-m


Taking a sample of 2000 emails randomly from each user

In [15]:
df_sample = emails_df[["Message-ID", "content", "user"]].groupby('user').apply(lambda df: df.sample(10))


In [16]:
df_sample["doc"] = df_sample.content.apply(load('en'))

Extracting character based features from each email

In [17]:
df_characters = pd.DataFrame()
    
for index, row in df_sample.iterrows():    
    num_chars = 0
    num_letters = 0
    num_upper_case = 0
    num_lower_case = 0
    num_punctuation = 0
    num_spaces = 0
    num_nums = 0
    
    for word in row['doc']:
        for char in str(word):
            num_chars += 1
            if char.isalpha():
                num_letters += 1
            if char.isupper():
                num_upper_case += 1
            if char.islower():
                num_lower_case += 1
            if char in punctuation:
                num_punctuation += 1
            if char.isspace():
                num_spaces += 1
            if char.isnumeric():
                num_nums += 1
            
    df_row = pd.DataFrame({"chars": num_chars, "letters": num_letters, "upper_case": num_upper_case, "lower_case": num_lower_case, "punctuations": num_punctuation, "spaces": num_spaces, "nums": num_nums}, index=[0])
    df_characters = df_characters.append(df_row, ignore_index=True)

df_sample['temp'] = ""
df_characters['temp'] = 1

for i in range(df_sample['user'].count()):
    df_sample['temp'][i] = i
    df_characters['temp'][i] = i
    
df_sample = pd.merge(df_sample, df_characters, on=['temp'])
#df_sample = df_sample.drop('temp', axis=1)  


Extracting word based features

In [18]:
df_words = pd.DataFrame()

for index, row in df_sample.iterrows():
    long_word = 5
    num_words = 0
    avg_letters_per_word = 0
    num_longwords = 0
    num_stopwords = 0
    num_error = 0
    TTR = 0
    hapaxes = []
    HTR = 0
    word_list = []
    sorted_word_frequency = []
    max_frequency = 0
    
    for word in row['doc']:
        if word.is_alpha:
            num_words += 1
            if len(word) > long_word:
                num_longwords += 1
            if word.is_stop:
                num_stopwords += 1
            if not word.vocab:
                num_error += 1
            word_list.append(str(word))
    
    if num_words > 0:
        avg_letters_per_word = float(row['letters']/num_words)
    else:
        avg_letters_per_word = 0
    
    if len(word_list) > 0:
        TTR = len(set(word_list))/len(word_list)
        hapaxes = list(filter(lambda x: word_list.count(x) == 1, word_list))
        HTR = len(hapaxes)/len(word_list)
        sorted_word_frequency = sorted(Counter(word_list).items(), key=itemgetter(1), reverse=True)
        max_frequency = sorted_word_frequency[0][1]
        
    df_row = pd.DataFrame({"words": num_words, "avg_letters_per_word": avg_letters_per_word, "longwords": num_longwords, "stopwords": num_stopwords, "spelling_errors": num_error, "TTR": TTR, "HTR": HTR, "max_frequency": max_frequency}, index=[0])
    df_words = df_words.append(df_row, ignore_index=True)

df_words['temp'] = 1

for i in range(df_sample['user'].count()):
    df_words['temp'][i] = i
    
df_sample = pd.merge(df_sample, df_words, on=['temp'])


C:\Users\faraa\anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Extracting sentence based features

In [19]:
df_sentences = pd.DataFrame()

for index, row in df_sample.iterrows():
    sentences = list(row['doc'].sents)
    num_sentences = len(sentences)
    
    #nr_words_l = [len(s) for s in doc.sents]
    avg_num_words_per_sentence = row['words']/num_sentences
    
    if len(sentences) > 1:
        std_num_words_per_sentence = stdev([len(sentence) for sentence in sentences])
    else:
        std_nr_word_per_sent = 0

    df_row = pd.DataFrame({"sentences": num_sentences, "avg_num_words_per_sentence": avg_num_words_per_sentence, "std_num_words_per_sentence": std_num_words_per_sentence}, index=[0])
    df_sentences = df_sentences.append(df_row, ignore_index=True)
    
df_sentences['temp'] = 1

for i in range(df_sample['user'].count()):
    df_sentences['temp'][i] = i
    
df_sample = pd.merge(df_sample, df_sentences, on=['temp'])

    

C:\Users\faraa\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Extracting punctuation based features

In [20]:
df_punctuations = pd.DataFrame()

for index, row in df_sample.iterrows():
    num_dots = 0
    num_commas = 0
    num_question_marks = 0
    num_exclamations = 0
    num_hyphens = 0
    num_colons = 0
    num_semicolons = 0
    
    for word in row['doc']:
        if str(word) == ".":
            num_dots += 1
        if str(word) == ",":
            num_commas += 1
        if str(word) == "?":
            num_question_marks += 1
        if str(word) == "!":
            num_exclamations += 1
        if str(word) == "-":
            num_hyphens += 1
        if str(word) == ":":
            num_colons += 1
        if str(word) == ";":
            num_semicolons += 1
        
    df_row = pd.DataFrame({"dots": num_dots, "commas": num_commas, "question_marks": num_question_marks, "exclamations": num_exclamations, "hyphens": num_hyphens, "colons": num_colons, "semicolons": num_semicolons}, index=[0])
    df_punctuations = df_punctuations.append(df_row, ignore_index=True)
    
df_punctuations['temp'] = 1

for i in range(df_sample['user'].count()):
    df_punctuations['temp'][i] = i
    
df_sample = pd.merge(df_sample, df_punctuations, on=['temp'])


Extracting paragraph based features

In [21]:
df_paragraphs = pd.DataFrame()

for index, row in df_sample.iterrows():
    sum_sent = 0
    av_sent = 0
    sum_word = 0
    av_word = 0

    paragraphs = row['content'].split('\n\n')
    paragraphs = [paragraph for paragraph in paragraphs if not (paragraph.isspace()or paragraph == "")]
#     for paragraph in paragraphs:
#         sentences = re.split('[?!.\n]', paragraph)
#         sentences = [sentence for sentence in sentences if not (sentence.isspace()or sentence == "")]
#         sum_sent += len(sentences)
#         for sentence in sentences:
#             words = sentence.split(" ")
#             words = [word for word in words if not (word.isspace()or word == "")]
#             sum_word += len(words)
    av_sent = row['sentences'] / len(paragraphs)
    av_word = row['words'] / len(paragraphs)

    df_row = pd.DataFrame({"n_paragraphs": len(paragraphs), "av_sent": av_sent, "av_word": av_word}, index=[0])
    df_paragraphs = df_paragraphs.append(df_row, ignore_index=True)

df_paragraphs['temp'] = 1

for i in range(df_sample['user'].count()):
    df_paragraphs['temp'][i] = i
    
df_sample = pd.merge(df_sample, df_paragraphs, on=['temp'])


C:\Users\faraa\anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
print (df_sample.head(5))

                                      Message-ID  \
0  <17342137.1075852710363.JavaMail.evans@thyme>   
1  <23603725.1075857640286.JavaMail.evans@thyme>   
2  <12889945.1075857632396.JavaMail.evans@thyme>   
3  <28178420.1075861673628.JavaMail.evans@thyme>   
4   <4799329.1075857598592.JavaMail.evans@thyme>   

                                             content      user  \
0  \n\n\nmost bullish thing at this point is movi...  arnold-j   
1  Gentlemen:\nThe following champagne is availab...  arnold-j   
2  no crap, what's your bid?\n\n\n\n\n"Eva Pao" <...  arnold-j   
3  We just finished doing the xpit that they aske...  arnold-j   
4  How about 4:00?\n\n\n\n\nSarah Wesner@ENRON\n0...  arnold-j   

                                                 doc temp  chars  letters  \
0  (\n\n\n, most, bullish, thing, at, this, point...    0    526      487   
1  (Gentlemen, :, \n, The, following, champagne, ...    1    530      401   
2  (no, crap, ,, what, 's, your, bid, ?, \n\n\n\n...    2  